In [ ]:
import requests 
from bs4 import BeautifulSoup as bs

class Parser():
    def __init__(self, company_name, nomin):
        self.company = company_name
        self.url = company_name
        self.nomin = nomin
        
    def get_responce(self, ):
        self.responce = requests.get(self.url)
        return self.responce
    
    def check(self, ):
        return self.responce.status_code == 200
    
    def get_industries(self, ):
        soup = bs(self.responce.text, 'html.parser')
        categories = []
        for category in soup.find_all('span', 'tm-company-profile__categories-wrapper'):
            categories.append(category.text.strip())
        return categories
    
    def get_about(self):
        soup = bs(self.responce.text, 'html.parser')
        return soup.find('span', 'tm-company-profile__content').text
    
    def get_date(self, ):
        soup = bs(self.responce.text, 'html.parser')
        return soup.find_all('dl', 'tm-description-list tm-description-list tm-description-list_variant-columns-info')[2].dd.time.text.strip()
    
    def get_rating(self, ):
        soup = bs(self.responce.text, 'html.parser')
        return float(soup.find('span', 'tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating').text.strip())
    
    def get_news_urls(self, ):
        news_url = self.url.replace('profile', 'news')
        responce = requests.get(news_url)
        soup = bs(responce.text, 'html.parser')
        urls = []
        for name in soup.find_all('h2', 'tm-title tm-title_h2'):
            try:
                urls.append(f'https://habr.com{name.a["href"]}')
            except:
                pass
        return urls
    
    def get_news_texts(self, ):
        urls = self.get_news_urls()
        texts = []
        for url in urls:
            r = requests.get(url)
            if r.status_code == 200:
                soup = bs(r.text, 'html.parser')
#                 try:
                for i in range(1, 5):
                    for t in soup.find_all('div', f'article-formatted-body article-formatted-body article-formatted-body_version-{i}'):
                        texts.append(t.text)
#                 except:
#                     pass
        return texts
    
    def get_all_info(self, ):
        print(f'! Start parsing {self.company} !')
        self.get_responce()
        if self.check:
            return {
                'name': self.company,
                'industries': self.get_industries(),
                'about': self.get_about(),
                'date': self.get_date(),
                'rating': self.get_rating(),
                'news': self.get_news_texts(), 
                'nomin': self.nomin
            } 
        else:
            return {}

        

In [ ]:
vk = Parser(company_name='https://habr.com/ru/companies/vk/profile/', nomin='vk').get_all_info()
# sber = Parser(company_name='sberbank').get_all_info()

In [ ]:
companies = [vk, sber]
data = {key: [] for key in vk.keys()}
for company in companies:
    for k, v in company.items():
        data[k].append(v)

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data)

In [ ]:
df